In [9]:
#making sure we train with our GPU, Pytorch is the fucking shit!!!
import torch
torch.cuda.is_available()

True

In [10]:
torch.cuda.device_count()

1

In [11]:
torch.cuda.current_device()

0

In [12]:
torch.cuda.device(0)

In [13]:
torch.cuda.get_device_name(0)

'NVIDIA TITAN RTX'

In [17]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import os
import warnings
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

# Set multiprocessing method
torch.multiprocessing.set_start_method('spawn', force=True)

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Custom Dataset class with exception handling
class BitcoinDataset(Dataset):
    def __init__(self, features, buy_labels, sell_labels, short_labels, closeshort_labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.buy_labels = torch.tensor(buy_labels, dtype=torch.long)
        self.sell_labels = torch.tensor(sell_labels, dtype=torch.long)
        self.short_labels = torch.tensor(short_labels, dtype=torch.long)
        self.closeshort_labels = torch.tensor(closeshort_labels, dtype=torch.long)

    def __len__(self):
        return len(self.buy_labels)
    
    def __getitem__(self, idx):
        try:
            features = self.features[idx]
            buy_label = self.buy_labels[idx]
            sell_label = self.sell_labels[idx]
            short_label = self.short_labels[idx]
            closeshort_label = self.closeshort_labels[idx]
            return features, buy_label, sell_label, short_label, closeshort_label
        except Exception as e:
            print(f"Error loading data at index {idx}: {e}")
            return torch.tensor([]), torch.tensor([]), torch.tensor([]), torch.tensor([]), torch.tensor([])  # Return empty tensors on error

# Load the preprocessed data
data_paths = {
    'minute1': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/minute1_trained.csv',
    'minute1_2': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/bitcoinm1_2_cleaned.csv',
    'minute5': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/minute5_trained.csv',
    'minute15': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/minute15_trained.csv',
    'minute30': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/minute30_trained.csv',
    'minute1_short': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/allshortingdata/minute1short_trained.csv',
    'minute1_2short': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/allshortingdata/bitcoinm1_2_cleanedshort.csv',
    'minute5short': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/allshortingdata/minute5short_trained.csv',
    'minute15short': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/allshortingdata/minute15short_trained.csv',
    'minute30short': 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/allshortingdata/minute30short_trained.csv'
}

# Function to load and concatenate datasets
def load_data(data_paths):
    data_frames = []
    for key, path in data_paths.items():
        df = pd.read_csv(path, dtype={'DateTime': str})  # Specify dtype for 'DateTime' column
        df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')  # Convert 'DateTime' to datetime
        df = df.set_index('DateTime')
        data_frames.append(df)
    combined_data = pd.concat(data_frames, axis=0)
    return combined_data

# Load and combine datasets
combined_data = load_data(data_paths)

# Debug: print the shape of the combined data
print(f"Combined data shape: {combined_data.shape}")

# Debug: check for NaNs in the data
nan_info = combined_data.isna().sum()
print("NaN values in each column:\n", nan_info)

# Fill or drop NaNs in critical columns
combined_data['Up_Or_Down'].fillna(0, inplace=True)
combined_data['Buy_Signal'].fillna(0, inplace=True)
combined_data['Sell_Signal'].fillna(0, inplace=True)
combined_data['Short_Signal'].fillna(0, inplace=True)
combined_data['CloseShort_Signal'].fillna(0, inplace=True)

# Drop rows where essential columns are still NaN
combined_data.dropna(subset=['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL', 'SPREAD'], inplace=True)

# Drop columns that are completely NaN
combined_data.drop(columns=['Unnamed: 14', 'Unnamed: 15'], inplace=True)

# Replace infinite values with NaN and then drop them
combined_data.replace([np.inf, -np.inf], np.nan, inplace=True)
combined_data.dropna(inplace=True)

# Debug: print the shape of the combined data after handling NaNs
print(f"Combined data shape after handling NaNs: {combined_data.shape}")

# Separate features and labels
features = combined_data.drop(columns=['Buy_Signal', 'Sell_Signal', 'Short_Signal', 'CloseShort_Signal']).values
buy_labels = combined_data['Buy_Signal'].values
sell_labels = combined_data['Sell_Signal'].values
short_labels = combined_data['Short_Signal'].values
closeshort_labels = combined_data['CloseShort_Signal'].values

# Debug: print the shapes of features and labels
print(f"Features shape: {features.shape}, Buy labels shape: {buy_labels.shape}, Sell labels shape: {sell_labels.shape}, Short labels shape: {short_labels.shape}, CloseShort labels shape: {closeshort_labels.shape}")

# Split the data
train_features, test_features, train_buy_labels, test_buy_labels, train_sell_labels, test_sell_labels, train_short_labels, test_short_labels, train_closeshort_labels, test_closeshort_labels = train_test_split(
    features, buy_labels, sell_labels, short_labels, closeshort_labels, test_size=0.2, shuffle=False)

train_features, val_features, train_buy_labels, val_buy_labels, train_sell_labels, val_sell_labels, train_short_labels, val_short_labels, train_closeshort_labels, val_closeshort_labels = train_test_split(
    train_features, train_buy_labels, train_sell_labels, train_short_labels, train_closeshort_labels, test_size=0.2, shuffle=False)

# Standardize the data
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

# Check data for NaNs and infinities
def check_data_for_nans(features, labels):
    if np.isnan(features).any() or np.isnan(labels).any():
        raise ValueError("Data contains NaNs")
    if np.isinf(features).any() or np.isinf(labels).any():
        raise ValueError("Data contains infinite values")

# Check training and validation data
check_data_for_nans(train_features, train_buy_labels)
check_data_for_nans(val_features, val_buy_labels)
check_data_for_nans(train_features, train_sell_labels)
check_data_for_nans(val_features, val_sell_labels)
check_data_for_nans(train_features, train_short_labels)
check_data_for_nans(val_features, val_short_labels)
check_data_for_nans(train_features, train_closeshort_labels)
check_data_for_nans(val_features, val_closeshort_labels)

# Create DataLoader function with conditional persistent_workers
def create_dataloader(features, buy_labels, sell_labels, short_labels, closeshort_labels, batch_size=12288, shuffle=True, num_workers=0):
    dataset = BitcoinDataset(features, buy_labels, sell_labels, short_labels, closeshort_labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True,
                            persistent_workers=num_workers > 0)
    return dataloader

# Define batch_size
batch_size = 12288  # Adjust to fit GPU memory

# Create DataLoaders for signals
train_loader = create_dataloader(train_features, train_buy_labels, train_sell_labels, train_short_labels, train_closeshort_labels, batch_size, num_workers=0)
val_loader = create_dataloader(val_features, val_buy_labels, val_sell_labels, val_short_labels, val_closeshort_labels, batch_size, num_workers=0)
test_loader = create_dataloader(test_features, test_buy_labels, test_sell_labels, test_short_labels, test_closeshort_labels, batch_size, num_workers=0)

print("Data Loaders created successfully!")

# Set the device for training (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Combined data shape: (2445944, 17)
NaN values in each column:
 OPEN                                1
HIGH                                1
LOW                                 1
CLOSE                               1
TICKVOL                             1
VOL                                 1
SPREAD                              1
Change_Open_Close                   1
Change_High_Low                     1
Up_Or_Down                    1078761
Profit_Between_Time_Series          1
Buy_Signal                     930747
Sell_Signal                    930747
Unnamed: 14                   2445944
Unnamed: 15                   2445943
Short_Signal                  1515198
CloseShort_Signal             1515198
dtype: int64
Combined data shape after handling NaNs: (2445943, 15)
Features shape: (2445943, 11), Buy labels shape: (2445943,), Sell labels shape: (2445943,), Short labels shape: (2445943,), CloseShort labels shape: (2445943,)
Data Loaders created successfully!
Using device: cuda


In [18]:
# Define the RNN model
import torch.nn as nn
import torch.optim as optim

# Define the RNN model
class BitcoinPriceLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(BitcoinPriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5_buy = nn.Linear(hidden_size, output_size)
        self.fc5_sell = nn.Linear(hidden_size, output_size)
        self.fc5_short = nn.Linear(hidden_size, output_size)
        self.fc5_closeshort = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))  # Added another activation
        out = torch.relu(self.fc3(out))  # Added another activation
        out = torch.relu(self.fc4(out))  # Added another activation
        buy_out = self.fc5_buy(out)
        sell_out = self.fc5_sell(out)
        short_out = self.fc5_short(out)
        closeshort_out = self.fc5_closeshort(out)
        return buy_out, sell_out, short_out, closeshort_out

# Model parameters
input_size = combined_data.shape[1] - 4  # Number of features (exclude 'Buy_Signal', 'Sell_Signal', 'Short_Signal', 'CloseShort_Signal')
hidden_size = 1024  # Reduced hidden size
num_layers = 6  # Reduced number of layers
output_size = 2  # Binary classification (up or down)

# Initialize the model, loss function, and optimizer
model = BitcoinPriceLSTM(input_size, hidden_size, num_layers, output_size, dropout=0.3).to(device)

# Initialize weights
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name or 'weight_hh' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'bias' in name:
                nn.init.constant_(param.data, 0)

model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)  # Adjusted learning rate
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

print("Model, loss function, and optimizer created successfully!")

Model, loss function, and optimizer created successfully!


In [19]:
# Function to count the number of parameters in a model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Print the number of parameters in the model
num_params = count_parameters(model)
print(f'The model has {num_params:,} trainable parameters')

The model has 50,438,152 trainable parameters


In [20]:
#Make sure we are training fully on our GPU

In [21]:
# Training function
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10):
    scaler = GradScaler()
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        running_loss = 0.0
        
        for features, buy_labels, sell_labels, short_labels, closeshort_labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
            features = features.to(device, non_blocking=True)
            buy_labels = buy_labels.to(device, non_blocking=True)
            sell_labels = sell_labels.to(device, non_blocking=True)
            short_labels = short_labels.to(device, non_blocking=True)
            closeshort_labels = closeshort_labels.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            with autocast():
                buy_out, sell_out, short_out, closeshort_out = model(features.unsqueeze(1))  # Add batch dimension for LSTM
                buy_loss = criterion(buy_out, buy_labels)
                sell_loss = criterion(sell_out, sell_labels)
                short_loss = criterion(short_out, short_labels)
                closeshort_loss = criterion(closeshort_out, closeshort_labels)
                loss = buy_loss + sell_loss + short_loss + closeshort_loss
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item() * features.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        val_loss = validate_model(model, val_loader, criterion)
        
        train_losses.append(epoch_loss)
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        
        epoch_time = time.time() - start_time
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {epoch_time:.2f}s")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping")
                model.load_state_dict(torch.load('best_model.pth'))
                break
    
    return train_losses, val_losses

def validate_model(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    
    with torch.no_grad():
        for features, buy_labels, sell_labels, short_labels, closeshort_labels in val_loader:
            features = features.to(device, non_blocking=True)
            buy_labels = buy_labels.to(device, non_blocking=True)
            sell_labels = sell_labels.to(device, non_blocking=True)
            short_labels = short_labels.to(device, non_blocking=True)
            closeshort_labels = closeshort_labels.to(device, non_blocking=True)
            
            with autocast():
                buy_out, sell_out, short_out, closeshort_out = model(features.unsqueeze(1))  # Add batch dimension for LSTM
                buy_loss = criterion(buy_out, buy_labels)
                sell_loss = criterion(sell_out, sell_labels)
                short_loss = criterion(short_out, short_labels)
                closeshort_loss = criterion(closeshort_out, closeshort_labels)
                loss = buy_loss + sell_loss + short_loss + closeshort_loss
            
            running_loss += loss.item() * features.size(0)
    
    val_loss = running_loss / len(val_loader.dataset)
    return val_loss

def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss over Epochs')
    plt.show()

num_epochs = 100

train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs)

plot_losses(train_losses, val_losses)


Epoch 1/100: 100%|████████████████████████████████████████████████████████████████| 128/128 [00:40<00:00,  3.17batch/s]


Epoch 1/100, Loss: 0.4078, Val Loss: 1.2713, Time: 49.60s


Epoch 2/100: 100%|████████████████████████████████████████████████████████████████| 128/128 [00:40<00:00,  3.17batch/s]


Epoch 2/100, Loss: 0.1883, Val Loss: 1.2774, Time: 49.57s


Epoch 3/100: 100%|████████████████████████████████████████████████████████████████| 128/128 [00:40<00:00,  3.19batch/s]


...

Epoch 6/100, Loss: 0.1739, Val Loss: 1.4613, Time: 49.68s


Epoch 7/100: 100%|████████████████████████████████████████████████████████████████| 128/128 [00:40<00:00,  3.18batch/s]


Epoch 7/100, Loss: 0.1732, Val Loss: 1.6291, Time: 49.45s


Epoch 8/100:  38%|████████████████████████▉                                        | 49/128 [00:15<00:25,  3.09batch/s]


KeyboardInterrupt: 

In [ ]:
#F1 Score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

def evaluate_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(inputs.unsqueeze(1))  # Add batch dimension for LSTM
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    test_loss = running_loss / len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')
    
    return all_labels, all_preds

# Evaluate on test_buy_loader or test_sell_loader depending on your focus
all_labels, all_preds = evaluate_model(model, test_buy_loader, criterion)

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
#Save the model:
import torch

# Save the entire model
torch.save(model, 'model.pth')

In [ ]:
# Save the model
torch.save(model.state_dict(), 'model.pth')
print("Model saved successfully!")

In [ ]:
#Backtesting--Getting our Trained model
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Define the LSTM model
class BitcoinPriceLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(BitcoinPriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))  # Added another activation
        out = torch.relu(self.fc3(out))  # Added another activation
        out = torch.relu(self.fc4(out))  # Added another activation
        out = self.fc5(out)
        return out

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model parameters
input_size = 11  # Adjust based on the number of features excluding 'Buy_Signal' and 'Sell_Signal'
hidden_size = 1024
num_layers = 6
output_size = 2

# Load the trained model
model_path = "C:/Users/gunne/Documents/PriceNeuralNetwork/model.pth"
model = BitcoinPriceLSTM(input_size, hidden_size, num_layers, output_size, dropout=0.5).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Load the backtest data
backtest_data_path = 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/1walk_cleaned.csv'
backtest_data = pd.read_csv(backtest_data_path, parse_dates=['DateTime'], index_col='DateTime')

# Feature engineering
backtest_data['Change_Open_Close'] = ((backtest_data['CLOSE'] - backtest_data['OPEN']) / backtest_data['OPEN']) * 100
backtest_data['Change_High_Low'] = ((backtest_data['HIGH'] - backtest_data['LOW']) / backtest_data['HIGH']) * 100
backtest_data['Up_Or_Down'] = (backtest_data['Change_Open_Close'].diff() > 0).astype(int)
backtest_data['Profit_Between_Time_Series'] = backtest_data['Change_Open_Close'].diff().fillna(0)
backtest_data['Buy_Signal'] = (backtest_data['Profit_Between_Time_Series'] > 0.165).astype(int)
backtest_data['Sell_Signal'] = backtest_data['Buy_Signal'].shift(-1).fillna(0).astype(int)

# Prepare features for model prediction
features = backtest_data.drop(columns=['Buy_Signal', 'Sell_Signal']).values
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Predicting buy signals
with torch.no_grad():
    inputs = torch.tensor(scaled_features, dtype=torch.float32).to(device)
    outputs = model(inputs.unsqueeze(1))
    predicted_signals = torch.argmax(outputs, dim=1).cpu().numpy()

backtest_data['Predicted_Buy_Signal'] = predicted_signals

# Backtesting with leverage and profit cap
initial_capital = 100
capital = initial_capital
leverage = 100  # 1:100 leverage
capital_history = []

# Profit cap parameter
profit_cap_percentage = 0.165  # Cap the profit at this percentage
target_capital = 200000  # Target capital to stop trading

# Trade log
trade_log = []

# Initialize variables
entry_price = None
positions = 0

for i in range(len(backtest_data)):
    date_time = backtest_data.index[i]
    current_price = backtest_data['CLOSE'].iloc[i]

    if capital >= target_capital:
        print(f"Target capital of {target_capital} reached at {date_time}, stopping trading.")
        break

    if backtest_data['Predicted_Buy_Signal'].iloc[i] == 1 and positions == 0:
        # Enter position
        positions = leverage * (capital / current_price)
        entry_price = current_price
        trade_log.append(f"Entered position at {date_time}, index {i}, price: {entry_price}, positions: {positions}, capital: {capital}")
    
    if positions != 0:
        profit_percentage = (current_price - entry_price) / entry_price * 100
        
        if profit_percentage >= profit_cap_percentage or profit_percentage <= -1:
            
            # Compound the capital if profit was made
            if profit_percentage >= profit_cap_percentage:
                capital *= (1 + (profit_percentage / 100) * leverage)
                
            trade_log.append(f"Exited position at {date_time}, index {i}, price: {current_price}, profit_percentage: {profit_percentage}, capital: {capital}")
            
            # Reset positions
            positions = 0
            entry_price = None
    
    # Update capital history
    capital_history.append(capital)

# Fill the remaining capital history with the final capital value
if len(capital_history) < len(backtest_data):
    capital_history.extend([capital] * (len(backtest_data) - len(capital_history)))

# Output trade log
for log in trade_log:
    print(log)

# Plotting the results
backtest_data['Capital'] = capital_history

plt.figure(figsize=(14, 7))

plt.subplot(2, 1, 1)
plt.plot(backtest_data.index, backtest_data['CLOSE'], label='Market Price')
plt.scatter(backtest_data.index[backtest_data['Predicted_Buy_Signal'] == 1], backtest_data['CLOSE'][backtest_data['Predicted_Buy_Signal'] == 1], marker='^', color='green', alpha=1, label='Buy Signal')
plt.scatter(backtest_data.index[backtest_data['Predicted_Buy_Signal'] == 0], backtest_data['CLOSE'][backtest_data['Predicted_Buy_Signal'] == 0], marker='v', color='red', alpha=1, label='Sell Signal')
plt.title('Backtest Trading Performance')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(backtest_data.index, backtest_data['Capital'], label='Capital')
plt.axhline(y=initial_capital, color='r', linestyle='--', label='Initial Capital')
plt.axhline(y=target_capital, color='g', linestyle='--', label='Target Capital')
plt.title('Capital Growth Over Time')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.legend()



plt.tight_layout()
plt.show()


# Calculate performance metrics
actual_signals = backtest_data['Buy_Signal'].values
precision = precision_score(actual_signals, predicted_signals)
recall = recall_score(actual_signals, predicted_signals)
f1 = f1_score(actual_signals, predicted_signals)
accuracy = accuracy_score(actual_signals, predicted_signals)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
#Backtesting-with forex
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Define the LSTM model
class BitcoinPriceLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(BitcoinPriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))  # Added another activation
        out = torch.relu(self.fc3(out))  # Added another activation
        out = torch.relu(self.fc4(out))  # Added another activation
        out = self.fc5(out)
        return out

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model parameters
input_size = 11  # Adjust based on the number of features excluding 'Buy_Signal' and 'Sell_Signal'
hidden_size = 1024
num_layers = 6
output_size = 2

# Load the trained model
model_path = "C:/Users/gunne/Documents/PriceNeuralNetwork/model.pth"
model = BitcoinPriceLSTM(input_size, hidden_size, num_layers, output_size, dropout=0.5).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Load the backtest data
backtest_data_path = 'C:/Users/gunne/OneDrive/Desktop/bitcoin/trained/1monthforex_cleaned.csv'
backtest_data = pd.read_csv(backtest_data_path, parse_dates=['DateTime'], index_col='DateTime')

# Feature engineering
backtest_data['Change_Open_Close'] = ((backtest_data['CLOSE'] - backtest_data['OPEN']) / backtest_data['OPEN']) * 100
backtest_data['Change_High_Low'] = ((backtest_data['HIGH'] - backtest_data['LOW']) / backtest_data['HIGH']) * 100
backtest_data['Up_Or_Down'] = (backtest_data['Change_Open_Close'].diff() > 0).astype(int)
backtest_data['Profit_Between_Time_Series'] = backtest_data['Change_Open_Close'].diff().fillna(0)
backtest_data['Buy_Signal'] = (backtest_data['Profit_Between_Time_Series'] > 0.165).astype(int)
backtest_data['Sell_Signal'] = backtest_data['Buy_Signal'].shift(-1).fillna(0).astype(int)

# Prepare features for model prediction
features = backtest_data.drop(columns=['Buy_Signal', 'Sell_Signal']).values
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Predicting buy signals
with torch.no_grad():
    inputs = torch.tensor(scaled_features, dtype=torch.float32).to(device)
    outputs = model(inputs.unsqueeze(1))
    predicted_signals = torch.argmax(outputs, dim=1).cpu().numpy()

backtest_data['Predicted_Buy_Signal'] = predicted_signals

# Backtesting with leverage and profit cap
initial_capital = 1000
capital = initial_capital
leverage = 100  # 1:100 leverage
capital_history = []

# Profit cap parameter
profit_cap_percentage = 0.165  # Cap the profit at this percentage
target_capital = 200000  # Target capital to stop trading

# Trade log
trade_log = []

# Initialize variables
entry_price = None
positions = 0

for i in range(len(backtest_data)):
    date_time = backtest_data.index[i]
    current_price = backtest_data['CLOSE'].iloc[i]

    if capital >= target_capital:
        print(f"Target capital of {target_capital} reached at {date_time}, stopping trading.")
        break

    if backtest_data['Predicted_Buy_Signal'].iloc[i] == 1 and positions == 0:
        # Enter position
        positions = leverage * (capital / current_price)
        entry_price = current_price
        trade_log.append(f"Entered position at {date_time}, index {i}, price: {entry_price}, positions: {positions}, capital: {capital}")
    
    if positions != 0:
        profit_percentage = (current_price - entry_price) / entry_price * 100
        
        if profit_percentage >= profit_cap_percentage or profit_percentage <= -1:
            
            # Compound the capital if profit was made
            if profit_percentage >= profit_cap_percentage:
                capital *= (1 + (profit_percentage / 100) * leverage)
                
            trade_log.append(f"Exited position at {date_time}, index {i}, price: {current_price}, profit_percentage: {profit_percentage}, capital: {capital}")
            
            # Reset positions
            positions = 0
            entry_price = None
    
    # Update capital history
    capital_history.append(capital)

# Fill the remaining capital history with the final capital value
if len(capital_history) < len(backtest_data):
    capital_history.extend([capital] * (len(backtest_data) - len(capital_history)))

# Output trade log
for log in trade_log:
    print(log)

# Plotting the results
backtest_data['Capital'] = capital_history

plt.figure(figsize=(14, 7))

plt.subplot(2, 1, 1)
plt.plot(backtest_data.index, backtest_data['CLOSE'], label='Market Price')
plt.scatter(backtest_data.index[backtest_data['Predicted_Buy_Signal'] == 1], backtest_data['CLOSE'][backtest_data['Predicted_Buy_Signal'] == 1], marker='^', color='green', alpha=1, label='Buy Signal')
plt.scatter(backtest_data.index[backtest_data['Predicted_Buy_Signal'] == 0], backtest_data['CLOSE'][backtest_data['Predicted_Buy_Signal'] == 0], marker='v', color='red', alpha=1, label='Sell Signal')
plt.title('Backtest Trading Performance')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(backtest_data.index, backtest_data['Capital'], label='Capital')
plt.axhline(y=initial_capital, color='r', linestyle='--', label='Initial Capital')
plt.axhline(y=target_capital, color='g', linestyle='--', label='Target Capital')
plt.title('Capital Growth Over Time')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.legend()



plt.tight_layout()
plt.show()


# Calculate performance metrics
actual_signals = backtest_data['Buy_Signal'].values
precision = precision_score(actual_signals, predicted_signals)
recall = recall_score(actual_signals, predicted_signals)
f1 = f1_score(actual_signals, predicted_signals)
accuracy = accuracy_score(actual_signals, predicted_signals)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
#Backtesting--Connecting with MetaTrader5 data with ICMarketsSC-Demo Bitcoin account 
# Install Required Packages
!pip install --upgrade pip
!pip install --force-reinstall --no-binary :all: tbb
!pip install MetaTrader5 pandas numpy torch matplotlib scikit-learn

# Imports and Model Definition
import MetaTrader5 as mt5
import pandas as pd
import datetime
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Define the LSTM model
class BitcoinPriceLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(BitcoinPriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = torch.relu(self.fc4(out))
        out = self.fc5(out)
        return out

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
model_path = "C:/Users/gunne/Documents/PriceNeuralNetwork/model.pth"
model = BitcoinPriceLSTM(input_size=11, hidden_size=1024, num_layers=6, output_size=2, dropout=0.5).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
else:
    print("MetaTrader 5 initialized successfully")

# Login to the demo account
account = 51806647  # Ensure the account number is an integer
password = "lKDtOjEK6$rp@f"
server = "ICMarketsSC-Demo"

# Debug print for login details
print(f"Attempting to login with Account: {account}, Password: {password}, Server: {server}")

authorized = mt5.login(login=account, password=password, server=server)
if not authorized:
    print(f"Failed to connect to account #{account}, error code: {mt5.last_error()}")
else:
    print(f"Connected to account #{account}")

In [ ]:
#Backtesting--Part 2
# Fetch historical data from MetaTrader 5
symbol = "BTCUSD"
timeframe = mt5.TIMEFRAME_M1
utc_from = datetime.datetime(2022, 6, 1)
utc_to = datetime.datetime(2022, 7, 1)
rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
if rates is None:
    print("Failed to fetch rates, error code:", mt5.last_error())
else:
    print(f"Fetched {len(rates)} rates")

# Create DataFrame from the rates
rates_frame = pd.DataFrame(rates)
rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
rates_frame.set_index('time', inplace=True)

# Feature engineering
rates_frame['Change_Open_Close'] = ((rates_frame['close'] - rates_frame['open']) / rates_frame['open']) * 100
rates_frame['Change_High_Low'] = ((rates_frame['high'] - rates_frame['low']) / rates_frame['high']) * 100
rates_frame['Up_Or_Down'] = (rates_frame['Change_Open_Close'].diff() > 0).astype(int)
rates_frame['Profit_Between_Time_Series'] = rates_frame['Change_Open_Close'].diff().fillna(0)
rates_frame['Buy_Signal'] = (rates_frame['Profit_Between_Time_Series'] > 0.165).astype(int)
rates_frame['Sell_Signal'] = rates_frame['Buy_Signal'].shift(-1).fillna(0).astype(int)

# Prepare features for model prediction
features = rates_frame.drop(columns=['Buy_Signal', 'Sell_Signal']).values
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Predicting buy signals
with torch.no_grad():
    inputs = torch.tensor(scaled_features, dtype=torch.float32).to(device)
    outputs = model(inputs.unsqueeze(1))
    predicted_signals = torch.argmax(outputs, dim=1).cpu().numpy()

rates_frame['Predicted_Buy_Signal'] = predicted_signals

# Backtesting with leverage and profit cap
initial_capital = 100
risk_per_trade = 10  # Risking $10 per trade
capital = initial_capital
leverage = 100  # 1:100 leverage
capital_history = []

# Profit cap parameter
profit_cap_percentage = 0.165  # Cap the profit at this percentage

# Trade log
trade_log = []

# Initialize variables
entry_price = None
positions = 0

for i in range(len(rates_frame)):
    date_time = rates_frame.index[i]
    current_price = rates_frame['close'].iloc[i]

    if rates_frame['Predicted_Buy_Signal'].iloc[i] == 1 and positions == 0:
        # Enter position
        positions = leverage * (risk_per_trade / current_price)
        entry_price = current_price
        trade_log.append(f"Entered position at index {i}, time {date_time}, price: {entry_price}, positions: {positions}, capital: {capital}")

    if positions != 0:
        profit_percentage = (current_price - entry_price) / entry_price * 100

        if profit_percentage >= profit_cap_percentage or profit_percentage <= -1:
            # Exit position due to reaching profit cap or stop loss
            profit = positions * (current_price - entry_price) * leverage
            capital += profit

            trade_log.append(f"Exited position at index {i}, time {date_time}, price: {current_price}, profit_percentage: {profit_percentage}, profit: {profit}, capital: {capital}")

            # Compound the capital if profit was made
            if profit_percentage >= profit_cap_percentage:
                capital += profit  # Add profit directly to capital

            # Reset positions
            positions = 0
            entry_price = None

    # Update capital history
    capital_history.append(capital)

# Output trade log
for log in trade_log:
    print(log)

# Plotting the results
rates_frame['Capital'] = capital_history

plt.figure(figsize=(14, 7))

plt.subplot(2, 1, 1)
plt.plot(rates_frame.index, rates_frame['close'], label='Market Price')
plt.scatter(rates_frame.index[rates_frame['Predicted_Buy_Signal'] == 1], rates_frame['close'][rates_frame['Predicted_Buy_Signal'] == 1], marker='^', color='green', alpha=1, label='Buy Signal')
plt.scatter(rates_frame.index[rates_frame['Predicted_Buy_Signal'] == 0], rates_frame['close'][rates_frame['Predicted_Buy_Signal'] == 0], marker='v', color='red', alpha=1, label='Sell Signal')
plt.title('Backtest Trading Performance')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(rates_frame.index, rates_frame['Capital'], label='Capital')
plt.axhline(y=initial_capital, color='r', linestyle='--', label='Initial Capital')
plt.axhline(y=200000, color='g', linestyle='--', label='Target Capital')
plt.title('Capital Growth Over Time')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
#Backtesting--Continuous testing, for each minute, 60 minutes:
import MetaTrader5 as mt5
import pandas as pd
import datetime
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import time

# Define the LSTM model
class BitcoinPriceLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(BitcoinPriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = torch.relu(self.fc4(out))
        out = self.fc5(out)
        return out

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
model_path = "C:/Users/gunne/Documents/PriceNeuralNetwork/model.pth"
model = BitcoinPriceLSTM(input_size=11, hidden_size=1024, num_layers=6, output_size=2, dropout=0.5).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
else:
    print("MetaTrader 5 initialized successfully")

In [3]:
#Backtesting
import MetaTrader5 as mt5
import pandas as pd
import datetime
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import time

# Define the LSTM model
class BitcoinPriceLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(BitcoinPriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = torch.relu(self.fc4(out))
        out = self.fc5(out)
        return out

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
model_path = "C:/Users/gunne/Documents/PriceNeuralNetwork/model.pth"
model = BitcoinPriceLSTM(input_size=11, hidden_size=1024, num_layers=6, output_size=2, dropout=0.5).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
else:
    print("MetaTrader 5 initialized successfully")

# Login to the demo account
account = 51806647  # Ensure the account number is an integer
password = "lKDtOjEK6$rp@f"
server = "ICMarketsSC-Demo"

# Debug print for login details
print(f"Attempting to login with Account: {account}, Password: {password}, Server: {server}")

authorized = mt5.login(login=account, password=password, server=server)
if not authorized:
    print(f"Failed to connect to account #{account}, error code: {mt5.last_error()}")
else:
    print(f"Connected to account #{account}")

MetaTrader 5 initialized successfully
Attempting to login with Account: 51806647, Password: lKDtOjEK6$rp@f, Server: ICMarketsSC-Demo
Connected to account #51806647


In [4]:
# Set up initial capital and leverage
# Set up initial capital and leverage
initial_capital = 100
risk_per_trade = 10  # Risk $10 per trade
leverage = 100  # 1:100 leverage
capital = initial_capital
profit_cap_percentage = 0.165
positions = 0
entry_price = None

# Trade log
trade_log = []

# Open trade log file in append mode
log_file_path = "trade_log.txt"

# Run the trading loop
while True:
    # Fetch live data
    rates = mt5.copy_rates_from_pos("BTCUSD", mt5.TIMEFRAME_M1, 0, 100)
    rates_frame = pd.DataFrame(rates)
    rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    rates_frame.set_index('time', inplace=True)

    # Feature engineering
    rates_frame['Change_Open_Close'] = ((rates_frame['close'] - rates_frame['open']) / rates_frame['open']) * 100
    rates_frame['Change_High_Low'] = ((rates_frame['high'] - rates_frame['low']) / rates_frame['high']) * 100
    rates_frame['Up_Or_Down'] = (rates_frame['Change_Open_Close'].diff() > 0).astype(int)
    rates_frame['Profit_Between_Time_Series'] = rates_frame['Change_Open_Close'].diff().fillna(0)
    rates_frame['Buy_Signal'] = (rates_frame['Profit_Between_Time_Series'] > 0.165).astype(int)
    rates_frame['Sell_Signal'] = rates_frame['Buy_Signal'].shift(-1).fillna(0).astype(int)

    # Prepare features for model prediction
    features = rates_frame.drop(columns=['Buy_Signal', 'Sell_Signal']).values
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Predicting buy signals
    with torch.no_grad():
        inputs = torch.tensor(scaled_features, dtype=torch.float32).to(device)
        outputs = model(inputs.unsqueeze(1))
        predicted_signals = torch.argmax(outputs, dim=1).cpu().numpy()

    rates_frame['Predicted_Buy_Signal'] = predicted_signals

    # Execute trades based on predictions
    current_price = rates_frame['close'].iloc[-1]

    if rates_frame['Predicted_Buy_Signal'].iloc[-1] == 1 and positions == 0:
        # Enter position
        positions = leverage * (risk_per_trade / current_price)
        entry_price = current_price
        log_entry = f"Entered position at time {datetime.datetime.now()}, price: {entry_price}, positions: {positions}, capital: {capital}"
        print(log_entry)
        with open(log_file_path, "a") as log_file:
            log_file.write(log_entry + "\n")
        trade_log.append(log_entry)

    if positions != 0:
        profit_percentage = (current_price - entry_price) / entry_price * 100

        if profit_percentage >= profit_cap_percentage or profit_percentage <= -1:
            # Exit position due to reaching profit cap or stop loss
            profit = positions * (current_price - entry_price) * leverage
            capital += profit
            log_exit = f"Exited position at time {datetime.datetime.now()}, price: {current_price}, profit_percentage: {profit_percentage}, profit: {profit}, capital: {capital}"
            print(log_exit)
            with open(log_file_path, "a") as log_file:
                log_file.write(log_exit + "\n")
            trade_log.append(log_exit)

            # Compound the capital if profit was made
            if profit_percentage >= profit_cap_percentage:
                capital += profit  # Add profit directly to capital

            # Reset positions
            positions = 0
            entry_price = None

    # Print details of each iteration
    iteration_log = f"Iteration at time {datetime.datetime.now()}, current price: {current_price}, positions: {positions}, capital: {capital}"
    print(iteration_log)
    with open(log_file_path, "a") as log_file:
        log_file.write(iteration_log + "\n")

    # Wait before the next iteration (e.g., 1 minute)
    time.sleep(60)

# Optional: Save trade log to a file (this is redundant if we already save logs during each trade)
with open(log_file_path, "a") as f:
    for log in trade_log:
        f.write(log + "\n")

Iteration at time 2024-05-26 06:53:16.281555, current price: 69056.86, positions: 0, capital: 100


KeyboardInterrupt: 